<a href="https://colab.research.google.com/github/asma-10/Identification-des-elements-non-vegetaux-et-des-organes-des-plantes-en-utilisant-YOLOv8/blob/main/Yolo_non_veg_kfold_and_hyperparams_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ultralytics
# Install and update Ultralytics and Ray Tune packages
#!pip install -U ultralytics "ray[tune]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.5/691.5 kB 3.8 MB/s eta 0:00:00


**Replacing class names in labeling files by integers**

In [ ]:

def replace_class_with_integer(file_path):
    # Define a mapping of class names to integers
    class_mapping = {'leaf': 0, 'flower': 1, 'fruit': 2, 'seed': 3, 'stem': 4,'root':5}

    # Read the content of the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Process each line and replace the class name with the corresponding integer
    modified_lines = []
    for line in lines:
        parts = line.strip().split()
        if parts and parts[0] in class_mapping:
            parts[0] = str(class_mapping[parts[0]])
            modified_lines.append(' '.join(parts))

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.write('\n'.join(modified_lines))

def process_folder(folder_path):
    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            replace_class_with_integer(file_path)

# Replace classes with integers in all text files in the specified folder
folder_path = '/content/drive/MyDrive/data/val/labels'
process_folder(folder_path)


**Training the YOLOv8 Model on data Using Kfold Cross Validation**

In [ ]:
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold

In [ ]:
dataset_path = Path('/content/drive/MyDrive/data/train') # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt")) # all data in 'labels'

In [ ]:
yaml_file = '/content/drive/MyDrive/data/dataset.yml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = sorted(classes.keys())

In [ ]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [ ]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

In [ ]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [ ]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

In [ ]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [ ]:
supported_extensions = ['.jpg', '.jpeg', '.png']

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images').rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

In [ ]:
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [ ]:
weights_path = '/content/yolov8n.pt'
model = YOLO(weights_path, task='detect')

In [ ]:
results = {}

# Define your additional arguments here
batch = 16
project = 'kfold_demo'
epochs = 2

for k in range(ksplit):
    dataset_yaml = ds_yamls[k]
    model.train(data=dataset_yaml,epochs=epochs, batch=batch, project=project)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis

Ultralytics YOLOv8.0.236 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=/content/yolov8n.pt, data=/content/drive/MyDrive/non veg detection.v6i.yolov8/train/2024-01-07_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=2, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold_demo, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

train: Scanning /content/drive/MyDrive/non veg detection.v6i.yolov8/train/2024-01-07_5-Fold_Cross-val/split_1/train/labels.cache... 579 images, 68 backgrounds, 0 corrupt: 100%|██████████| 647/647 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/non veg detection.v6i.yolov8/train/2024-01-07_5-Fold_Cross-val/split_1/val/labels.cache... 114 images, 46 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<?, ?it/s]


Plotting labels to kfold_demo/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        2/2         0G     0.7745      1.145     0.9926         21        640: 100%|██████████| 41/41 [08:54<00:00, 13.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:48<00:00,  8.13s/it]

                   all        163        546      0.528      0.637      0.531      0.392



2 epochs completed in 0.329 hours.
Optimizer stripped from kfold_demo/train22222/weights/last.pt, 6.2MB
Optimizer stripped from kfold_demo/train22222/weights/best.pt, 6.2MB

Validating kfold_demo/train22222/weights/best.pt...
Ultralytics YOLOv8.0.236 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:47<00:00,  7.89s/it]


                   all        163        546      0.528      0.637       0.53      0.392
      M database label        163         98      0.573      0.837      0.582      0.399
      accession number        163          8      0.167      0.125      0.151     0.0595
   institutional label        163        124      0.494      0.911      0.646      0.554
         original data        163         43      0.348      0.397      0.422      0.216
                 scale        163         97      0.559      0.954       0.65      0.517
                 stamp        163         35      0.352      0.571       0.37      0.263
                swatch        163        116      0.607          1      0.706      0.657
             swing tag        163         10      0.652       0.94      0.855      0.537
                 taxon        163         15          1          0      0.392      0.322
Speed: 3.3ms preprocess, 252.7ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to kfold_dem

In [ ]:
from sklearn.model_selection import ParameterGrid
import os

# Define hyperparameters to tune
learning_rates = [0.0001, 0.001, 0.01]
batch_sizes = [32, 64, 128]
optimizers = ['adam', 'sgd']
epochs = [50, 100, 150]
network_architectures = ['yolov8', 'yolov8-tiny']
activation_functions = ['relu', 'leaky_relu']
loss_functions = ['binary_crossentropy', 'focal_loss']
data_augmentation = [True, False]

# Set up the search space
search_space = {'learning_rate': learning_rates,
                'batch_size': batch_sizes,
                'optimizer': optimizers,
                'epochs': epochs,
                'network_architecture': network_architectures,
                'activation_function': activation_functions,
                'loss_function': loss_functions,
                'data_augmentation': data_augmentation}

# Create parameter grid
parameter_grid = ParameterGrid(search_space)

# Train and evaluate the model for each combination of hyperparameters
for parameters in parameter_grid:
    # Set hyperparameters for the model
    learning_rate = parameters['learning_rate']
    batch_size = parameters['batch_size']
    optimizer = parameters['optimizer']
    num_epochs = parameters['epochs']
    architecture = parameters['network_architecture']
    activation = parameters['activation_function']
    loss = parameters['loss_function']
    augmentation = parameters['data_augmentation']

    # Train and evaluate the model
    train_yolo_v8(learning_rate, batch_size, optimizer, num_epochs, architecture, activation, loss, augmentation)
    mAP = evaluate_yolo_v8()

    # Save the results
    result = {'learning_rate': learning_rate,
              'batch_size': batch_size,
              'optimizer': optimizer,
              'num_epochs': num_epochs,
               'architecture': architecture,
              'activation':activation,
              'loss': loss,
              'augmentation':augmentation}